In [1]:
from pathlib import Path
import geopandas as gpd
import itertools
import cv2
import numpy as np
from tqdm import tqdm
from random import shuffle
image_root = Path('./downloads/')
data_root = Path('./data/3DMassingShapefile_2019_WGS84')
output_loc = f'./image_data_singles/'
output_size = (224,224)

# Load in Massings data and locate image directories.

In [2]:
gdf = gpd.read_file(data_root)

In [10]:
dirs = [d.parts[-1] for d in image_root.iterdir() if d.is_dir()]

# Main method: loop over massings data. If corresponding images exist, resize them to the desired size and add a row with the corresponding data to the output dataframe.

In [11]:
df = []
for k,v in tqdm(gdf.iterrows(),total=len(dirs)):
    if str(k) in dirs:
        directory = Path.joinpath(image_root,str(k))
        count = len([t for t in directory.iterdir() if t.suffix == '.jpg'])
        images = []
        for image in directory.iterdir():
            if image.suffix == '.jpg':
                images.append(cv2.imread(image.as_posix()))
        if len(images) == 0:
            continue
        for i,image in enumerate(images):
            image = cv2.resize(image, output_size)
            filename = f'{output_loc}{k}_{i}.jpg'
            cv2.imwrite(filename, image)
            t = {'filename':filename}
            for col in ['MIN_HEIGHT', 'MAX_HEIGHT', 'AVG_HEIGHT', 'HEIGHT_MSL', 'SURF_ELEV', 'LONGITUDE', 'LATITUDE', 'POLY_AREA','PERIMETER']:
                t[col] = v[col]
            df.append(t)

423008it [14:52, 473.85it/s]                           


In [12]:
import pandas as pd
df = pd.DataFrame(df)

# Save to output file. Additionally, preview file

In [13]:
df.to_pickle('labels.pickle')

In [14]:
df

,filename,MIN_HEIGHT,MAX_HEIGHT,AVG_HEIGHT,HEIGHT_MSL,SURF_ELEV,LONGITUDE,LATITUDE,POLY_AREA,PERIMETER
0,./image_data_singles/0_0.jpg,0.0000,9.3506,7.309000,165.57000,158.260969,-79.592266,43.660304,3643.971349,249.201332
1,./image_data_singles/0_1.jpg,0.0000,9.3506,7.309000,165.57000,158.260969,-79.592266,43.660304,3643.971349,249.201332
2,./image_data_singles/0_2.jpg,0.0000,9.3506,7.309000,165.57000,158.260969,-79.592266,43.660304,3643.971349,249.201332
3,./image_data_singles/0_3.jpg,0.0000,9.3506,7.309000,165.57000,158.260969,-79.592266,43.660304,3643.971349,249.201332
4,./image_data_singles/1_0.jpg,0.0000,12.8670,8.204000,164.81000,156.605735,-79.588970,43.663573,2742.372728,304.217764
...,...,...,...,...,...,...,...,...,...,...
81302,./image_data_singles/422722_0.jpg,0.0000,10.3290,7.984900,117.09689,109.111991,-79.133790,43.787431,229.159647,62.189127
81303,./image_data_singles/422722_1.jpg,0.0000,10.3290,7.984900,117.09689,109.111991,-79.133790,43.787431,229.159647,62.189127
81304,./image_data_singles/422787_0.jpg,3.5551,20.0470,4.290567,104.66281,100.372246,-79.135387,43.799908,379.169546,89.062114
81305,./image_data_singles/422853_0.jpg,0.0000,23.4400,5.773000,114.79772,109.024719,-79.138031,43.799728,424.734152,86.357988
